In [20]:
import os
import configparser
from langchain_groq import ChatGroq
from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage, SystemMessage

config = configparser.ConfigParser()
config.read('../config.ini')

groq = config['groq']
cohere = config['cohere']

os.environ['GROQ_API_KEY'] = groq.get('GROQ_API_KEY')
os.environ['COHERE_API_KEY'] = cohere.get('COHERE_API_KEY')


In [21]:
import random
def actor_selector():
    actor=["Amithabh Bachchan","Tom Cruise","Allu Arjun","Salman Khan","Robert Downey Jr"]
    return random.choice(actor)

def location_selector():
    locations = ["Norwey", "Kerala", "Dubai", "Chaina","Japan"]
    return random.choice(locations)

def theme_selector():
    themes = ["Adventure", "Horror", "Comedy", "Mystery","Sci-Fi"]
    return random.choice(themes)

In [30]:

from langchain_core.chat_history import (
    #BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
import asyncio
store = {}

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:  # If a new session then create a new memory store.
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

async def Message_to_Model(model,session_id,prompt):

    conversation_history=get_session_history(session_id)

    conversation_history.add_message(HumanMessage(content=prompt))
    
    response=model.invoke(
        [SystemMessage(content="You are a Movie Story Writer"),
         HumanMessage(content=prompt)]
    )

    conversation_history.add_message(SystemMessage(content=response.content))
    
    return response.content
   


In [31]:
async def model1_cohere(actor,location,theme):
    session_id = "cohere_story"
    prompt = f"Create a Bollywood story starring {actor}, set in {location}, with a theme of {theme}."
    cohere_model = ChatCohere(model="command-r-plus")
    return await Message_to_Model(cohere_model, session_id, prompt)

async def model2_groq(actor, location, theme):
    session_id = "groq_story"
    prompt = f"Create a Hollywood story starring {actor}, set in {location}, with a theme of {theme}."
    groq_model = ChatGroq(model="llama3-8b-8192")
    return await Message_to_Model(groq_model, session_id, prompt)

In [32]:
actor=actor_selector()
location=location_selector()
theme=theme_selector()

print(f"actor is {actor}")
print(f"location is {location}")
print(f"theme is {theme}")

actor is Salman Khan
location is Japan
theme is Adventure


In [33]:
cohere_story = await model1_cohere(actor, location, theme)
print(f"\nStory from Cohere Model:\n{cohere_story}\n")

groq_story = await model2_groq(actor, location, theme)
print(f"\nStory from Groq Model:\n{groq_story}\n")


Story from Cohere Model:
Title: "Tokyo Tiger"

Introduction:

Salman Khan stars as Raj, a free-spirited and adventurous Indian man who finds himself in Tokyo, Japan, under unusual circumstances. Raj, always up for a thrilling experience, embarks on a journey through the vibrant and exotic land of Japan, where he uncovers a hidden mystery and finds love in the most unexpected place.

Storyline:

Act 1:

Raj arrives in Tokyo, captivated by the bustling city's vibrant culture and futuristic landscape. He befriends a local tour guide, Aki (a skilled martial artist and avid adventurer herself), who offers to show him the real Japan beyond the tourist hotspots. Together, they explore the vibrant streets of Shibuya and the serene beauty of Kyoto's temples.

During their adventures, Raj and Aki stumble upon an ancient Japanese legend about a hidden treasure—a priceless artifact known as the "Tokyo Tiger," said to bring immense fortune and power to its possessor. The legend speaks of clues hid